In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re
from statsbombpy import sb
import src as src
from src import normalizacion_nombres

In [ ]:
# Crear URLs para cada jornada
contador = 0
plantilla_url = "https://www.ceroacero.es/edicion/campeonato-espanhol-2015-16/87618?jornada_in={x}&fase=82379"
urls_por_jornada = {}
for contador in range(0,38):
    url = plantilla_url.format(x=contador+1)
    urls_por_jornada[contador+1] = url

In [ ]:
# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
df_jornadas = pd.DataFrame()
for x, i in urls_por_jornada.items():
    # URL
    base_url = i
    driver.get(base_url)
    datos = []
    encabezado = []
    # Espera
    wait = WebDriverWait(driver, 10)
    # Clic en el botón de cookies
    try:
        cookie_button = driver.find_element(By.CSS_SELECTOR, ".fc-button-label")
        cookie_button.click()
    except:
        pass  # En caso de que no aparezca seguir con el código
    try:
        cookie_button2 = driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16")
        cookie_button2.click()
    except:
        pass  # En caso de que no aparezca seguir con el código

    # Buscar clasificacion
    tabla_element = wait.until(EC.presence_of_element_located((By.ID, "DataTables_Table_0")))
    if not encabezado: # Solo la primera vez
        headers_th = tabla_element.find_elements(By.XPATH, ".//thead/tr/th")
        # Priorizar el texto del 'aria-label' si el texto normal está vacío o es genérico
        # O simplemente tomar el texto visible
        for th in headers_th:
            aria_label = th.get_attribute("aria-label")
            text = th.text.strip()
            if text and text not in [":", ""]: # Usar texto si es significativo
                encabezado.append(text)
            elif aria_label and ":" in aria_label: # Procesar aria-label
                    # "Pts: activate to sort column ascending" -> "Pts"
                encabezado.append(aria_label.split(":")[0].strip())
            elif text: # Fallback a texto si aria-label no es útil
                encabezado.append(text)
            else: # Si ambos son malos, un placeholder o intentar inferir
                encabezado.append(f"Col_{len(encabezado)+1}") # Placeholder

        # Limpieza de posibles columnas vacías o no deseadas del final si tienen placeholders
        if encabezado and encabezado[-1].startswith("Col_"):
            # La última columna parece ser de acciones, la removemos si no tiene un buen nombre
                is_action_col = tabla_element.find_elements(By.XPATH, ".//tbody/tr[1]/td[last()]/a/span[contains(@class, 'icn_zerozero')]")
                if is_action_col:
                    encabezado.pop()

        # Asegurarse de que los nombres de las columnas sean únicos si es necesario
        # y que coincidan con el número de celdas que planeas extraer.
        # Por la estructura de la tabla HTML, el nombre del equipo está en la 3ra columna visual,
        # pero la 2da <th> puede estar vacía (para el logo).
        # Ajuste manual si es necesario, basado en la inspección visual de la tabla:
        # Ejemplo de ajuste si los nombres automáticos no son perfectos:
        expected_cols = ["Pos", "Logo", "Equipo", "Pts", "P", "V", "E", "D", "GM", "GR", "DG"] # Menos la última de acción
        if len(encabezado) >= len(expected_cols):
                # La primera columna <th> es para Pos, la segunda para Logo (sin texto), la tercera para Equipo (texto vacío)
                # Usaremos una lista predefinida para mayor robustez con esta tabla específica.
                column_headers = expected_cols


    # Extraer filas de datos
    filas = tabla_element.find_elements(By.XPATH, ".//tbody/tr")
    #print(f"Encontradas {len(filas)} filas en Jornada {i+1}.")

    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        if len(celdas) >= len(column_headers): # Asegurar que hay suficientes celdas
            # Extraer el texto de cada celda. Adaptar según la estructura.
            # La columna del equipo (índice 2) tiene un enlace <a>
            posicion = celdas[0].text.strip()
            # logo_img_src = celdas[1].find_element(By.TAG_NAME, "img").get_attribute("src") # Opcional
            equipo_link = celdas[2].find_element(By.TAG_NAME, "a")
            equipo = equipo_link.text.strip()
            # equipo_url = equipo_link.get_attribute("href") # Opcional

            # Resto de las celdas (Pts, P, V, E, D, GM, GR, DG)
            # Índices 3 a 10 para los datos si column_headers tiene 11 elementos
            datos_numericos = [celda.text.strip() for celda in celdas[3:len(column_headers)]]


            registro = [posicion, "logo_placeholder", equipo] + datos_numericos
            registro_dict = dict(zip(column_headers, registro))
            registro_dict['Jornada'] = x # Añadir la jornada actual
            datos.append(registro_dict)
        #else:
            #print(f"Fila omitida en Jornada {i+1} por no tener suficientes celdas: {[c.text for c in celdas]}")


    # 5. Convertir a DataFrame de Pandas y guardar (opcional)
    if datos:
        df = pd.DataFrame(datos)
        df_jornadas = pd.concat([df_jornadas, df], ignore_index=True)



# 4. Cerrar el WebDriver
driver.quit()


In [ ]:
df_jornadas.drop(columns=(["Logo"]), inplace=True)

In [ ]:
df_wyscout = pd.read_csv("data/Tabla_Partidos.csv")
clubs_statsbomb = df_wyscout["Local"].unique().tolist()
clubs_jornadas = df_jornadas["Equipo"].unique().tolist()
df_clubes = src.normalizacion_nombres(clubs_statsbomb, clubs_jornadas)
# Crear diccionario
dict_nombres_equipos = dict(zip(df_clubes["Nombre_unificado"], df_clubes["Nombre_original"]))
# Mapear nombres en todas las columnas
df_jornadas["Equipo"] = df_jornadas["Equipo"].map(dict_nombres_equipos)
df_combinado = df_jornadas.merge(
    df_wyscout[["Local", "id_club_local_L"]],
    left_on=["Equipo"],
    right_on=["Local"],
    how="left"
)
df_combinado.drop(columns=(["Local"]), inplace=True)
df_combinado.rename(columns=({"id_club_local_L":"clubid"}), inplace=True)

In [12]:
df_combinado.to_csv("data/clasificacion.csv" ,index=False)